# Environment Verification

In [1]:
import torch
import ultralytics

print("Ultralytics version:", ultralytics.__version__)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

!nvidia-smi

Ultralytics version: 8.3.221
PyTorch version: 2.5.1
CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU
Tue Oct 28 00:15:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    Off |   00000000:01:00.0 Off |                  N/A |
| N/A   54C    P3             12W /   60W |      15MiB /   6141MiB |      0%      Default |
|                          

# Roboflow API Setup

In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key="iSUpQqSxeaupKuugZKmc")

project = rf.workspace("metis-bwgip").project("metis_detection_yolo11-ashhg")
dataset = project.version(1).download("yolov11", location="../data/raw")

loading Roboflow workspace...
loading Roboflow project...


# YOLOv11 Exploration

In [3]:
from ultralytics import YOLO
from pathlib import Path
import shutil

model_n = YOLO('yolo11n.pt')
model_s = YOLO('yolo11s.pt')

In [6]:
data_folder = Path("../data/raw/test/images")
results_folder = Path('../results')

if results_folder.exists():
    shutil.rmtree(results_folder)

for file_path in data_folder.iterdir():
    if file_path.is_file():
        model_n.predict(file_path, show=False, save=True, project=results_folder, name='yolo11n_predictions', exist_ok=True)
        model_s.predict(file_path, show=False, save=True, project=results_folder, name='yolo11s_predictions', exist_ok=True)


image 1/1 /home/dcortesav/proj-metis/notebooks/../data/raw/test/images/IMG_20251006_133057_jpg.rf.a8776f7b871e08df9c746931ca49976e.jpg: 640x640 1 suitcase, 3.1ms
Speed: 1.1ms preprocess, 3.1ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/dcortesav/proj-metis/results/yolo11n_predictions

image 1/1 /home/dcortesav/proj-metis/notebooks/../data/raw/test/images/IMG_20251006_133057_jpg.rf.a8776f7b871e08df9c746931ca49976e.jpg: 640x640 1 handbag, 5.8ms
Speed: 1.1ms preprocess, 5.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/dcortesav/proj-metis/results/yolo11s_predictions

image 1/1 /home/dcortesav/proj-metis/notebooks/../data/raw/test/images/IMG_20251006_135731_jpg.rf.401e62eb86e0643e7f25e3aa58f205e3.jpg: 640x640 1 snowboard, 1 surfboard, 3.2ms
Speed: 1.1ms preprocess, 3.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /home/dcortesav/proj-metis/results/yolo11n_predic

# Training Exploration

In [8]:
data_path = "../data/raw/data.yaml"
results = model_n.train(data=data_path, batch=-1, epochs=100, project="../training_results", name="yolo11n_train",imgsz=512)

Ultralytics 8.3.221 🚀 Python-3.10.13 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 5771MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data/raw/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo11n_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.

In [9]:
trained_model = YOLO('../training_results/yolo11n_train/weights/best.pt')

In [10]:
data_folder = Path("../data/raw/test/images")
results_folder = Path('../test_results')

if results_folder.exists():
    shutil.rmtree(results_folder)

for file_path in data_folder.iterdir():
    if file_path.is_file():
        trained_model.predict(file_path, show=False, save=True, project=results_folder, name='tunned_model_predictions', exist_ok=True)


image 1/1 /home/dcortesav/proj-metis/notebooks/../data/raw/test/images/IMG_20251006_133057_jpg.rf.a8776f7b871e08df9c746931ca49976e.jpg: 512x512 (no detections), 3.3ms
Speed: 0.5ms preprocess, 3.3ms inference, 0.3ms postprocess per image at shape (1, 3, 512, 512)
Results saved to /home/dcortesav/proj-metis/test_results/tunned_model_predictions

image 1/1 /home/dcortesav/proj-metis/notebooks/../data/raw/test/images/IMG_20251006_135731_jpg.rf.401e62eb86e0643e7f25e3aa58f205e3.jpg: 512x512 1 brisa_coca-cola, 3.7ms
Speed: 0.7ms preprocess, 3.7ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 512)
Results saved to /home/dcortesav/proj-metis/test_results/tunned_model_predictions

image 1/1 /home/dcortesav/proj-metis/notebooks/../data/raw/test/images/IMG_5684_jpeg.rf.ec77d3695a967a9a07625143dcffc539.jpg: 512x512 1 cristal_postobon, 6.2ms
Speed: 0.9ms preprocess, 6.2ms inference, 0.6ms postprocess per image at shape (1, 3, 512, 512)
Results saved to /home/dcortesav/proj-metis/test_

# CPU M-Model Trainining

In [ ]:
model_m = YOLO('yolo11m.pt')

In [ ]:
model_m.train(data='small_training_set/data.yaml')